Takes in .AWD, sleep log if it exists, calendar, and spits out some pretty pictures

In [1]:
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, '/data/MoodGroup/actigraphy/gavi/actigraPy') 

import actigraPy.actigraPy as act
import importlib
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta

/data/MoodGroup/actigraphy/gavi/actigraPy/actigraPy/actigraPy.py:23: MatplotlibDeprecationWarning: 
The matplotlib.backends.tkagg module was deprecated in Matplotlib 3.0 and will be removed in 3.2.
  import matplotlib.backends.tkagg as tkagg


## Subject and directory info

In [2]:
# data and log directories
sub = '135' #subject NUMBER as STRING
sub_long = 'sub-MOA'+sub

out_dir = '/data/MoodGroup/actigraphy/KMOA/derivatives/preproc/' + sub_long#output directory
data_dir = '/data/MoodGroup/actigraphy/KMOA/raw/' + sub_long

if not os.path.exists(out_dir):
    os.mkdir(out_dir)


In [3]:
fn = {}
#.AWD
awd_fn = os.path.join(data_dir, '%s_0.AWD'%sub_long)
#sleeplog
fn['cal']= os.path.join(data_dir, '%s_calendar_log.xls'%sub_long)
#calendar
fn['log'] = os.path.join(data_dir, '%s_sleeplog.xls'%sub_long)
logs = []

if os.path.isfile(awd_fn):
    for name in fn.keys():
        if os.path.isfile(fn[name]):
            print("%s has %s file at %s"%(sub,name,fn[name]))
            logs.append(fn[name])
        else:
            print("no " + name)
            fn[name] = ''
else:
    print("DO NOT CONTINUE!! THERE IS NO AWD FILE")

135 has cal file at /data/MoodGroup/actigraphy/KMOA/raw/sub-MOA135/sub-MOA135_calendar_log.xls
135 has log file at /data/MoodGroup/actigraphy/KMOA/raw/sub-MOA135/sub-MOA135_sleeplog.xls


In [4]:
awd_fn

'/data/MoodGroup/actigraphy/KMOA/raw/sub-MOA135/sub-MOA135_0.AWD'

### Make Mtimes file

Figure out where to clip awd file

In [5]:
#read AWD file
awd_dat = act.read_AWD(awd_fn)
idx={'start':[0],'end':[len(awd_dat['dt_list'])-1]}
#get the start and stops from each log if they exist
for log in logs:
    log_dat, kw_dat, comments = act.read_log(log,awd_dat)
    
    if 'watch_on' in kw_dat.keys():
        on_date = kw_dat['watch_on'].iloc[0]['OnDate']
        on_time = kw_dat['watch_on'].iloc[0]['OnTime']
        on = datetime(on_date.year,on_date.month,on_date.day,on_time.hour,on_time.minute)
        #check if that time is in dt_list:
        if awd_dat['dt_list'].count(on) > 0:
            on_idx=awd_dat['dt_list'].index(on)
            idx['start'].append(on_idx)

    if 'watch_off' in kw_dat.keys():
        off_date = kw_dat['watch_off'].iloc[0]['OffDate']
        off_time = kw_dat['watch_off'].iloc[0]['OffTime']
        off = datetime(off_date.year,off_date.month,off_date.day,off_time.hour,off_time.minute)
        if awd_dat['dt_list'].count(off) > 0:
            off_idx=awd_dat['dt_list'].index(off)
            idx['end'].append(off_idx)
 
idx

      OffDate   OffTime     OnDate    OnTime      Comment  Unnamed: 5  \
0         NaT       NaN 2016-02-08  10:00:00        start         NaN   
1  2016-02-08  13:00:00 2016-02-08  14:30:00           3T         NaN   
2  2016-02-08  21:30:00 2016-02-09  07:00:00  Sleep study         NaN   
3  2016-02-09  14:00:00 2016-02-09  16:00:00           7T         NaN   
4  2016-02-10  10:00:00 2016-02-10  12:00:00          MEG         NaN   
5  2016-02-10  21:30:00 2016-02-11  07:00:00  Sleep study         NaN   
6  2016-02-11  09:00:00 2016-02-11  09:40:00     Infusion         NaN   
7  2016-02-11  15:00:00 2016-02-11  17:00:00          MEG         NaN   
8  2016-02-11  21:30:00 2016-02-12  07:00:00  Sleep study         NaN   
9  2016-02-12  08:00:00 2016-02-12  10:00:00           7T         NaN   
10 2016-02-13  10:00:00 2016-02-13  11:30:00           3T         NaN   
11 2016-02-21  10:00:00 2016-02-21  12:00:00           7T         NaN   
12 2016-02-22  13:00:00 2016-02-22  14:30:00       

{'start': [0, 144], 'end': [45947, 45283, 44479]}

In [6]:
start = max(idx['start'])
end = min(idx['end'])
print('start = %d, end = %d'%(start,end))

start = 144, end = 44479


## <font color='red'>Make sure the starts and ends make sense before clipping the data</font>


In [7]:
#modify these according to output from above!!
lim = [awd_dat['DateTime'][start],awd_dat['DateTime'][end]]
clip_dat = act.clip_dat(lim,awd_dat)

Now just run these cells in order to extract comments and write Mtimes

In [8]:
importlib.reload(act)
#Make master mk_idx
mk_idx,comments=act.get_markers(clip_dat)

In [29]:
#Read logs
mc_dict = {} #for Mtimes
for name in fn.keys():
    if os.path.isfile(fn[name]):
        c_dict={}
        mk_idx_tmp,comments_tmp = act.get_markers(clip_dat,fn[name])
        mk_idx[name]=mk_idx_tmp['l']
        
        #process comments
        zipped_pairs = zip(comments_tmp[0].tolist(),comments_tmp[1])
        c_dict['comments'] = [x for _, x in sorted(zipped_pairs)]
        c_dict['idxs'] = np.sort(comments_tmp[0]).tolist()
        mc_dict[name]=c_dict

      OffDate   OffTime     OnDate    OnTime      Comment  Unnamed: 5  \
0         NaT       NaN 2016-02-08  10:00:00        start         NaN   
1  2016-02-08  13:00:00 2016-02-08  14:30:00           3T         NaN   
2  2016-02-08  21:30:00 2016-02-09  07:00:00  Sleep study         NaN   
3  2016-02-09  14:00:00 2016-02-09  16:00:00           7T         NaN   
4  2016-02-10  10:00:00 2016-02-10  12:00:00          MEG         NaN   
5  2016-02-10  21:30:00 2016-02-11  07:00:00  Sleep study         NaN   
6  2016-02-11  09:00:00 2016-02-11  09:40:00     Infusion         NaN   
7  2016-02-11  15:00:00 2016-02-11  17:00:00          MEG         NaN   
8  2016-02-11  21:30:00 2016-02-12  07:00:00  Sleep study         NaN   
9  2016-02-12  08:00:00 2016-02-12  10:00:00           7T         NaN   
10 2016-02-13  10:00:00 2016-02-13  11:30:00           3T         NaN   
11 2016-02-21  10:00:00 2016-02-21  12:00:00           7T         NaN   
12 2016-02-22  13:00:00 2016-02-22  14:30:00       

In [24]:
(mc_dict['log']['comments'])

['shower',
 '7T MRI',
 'shower',
 'shower',
 'MEG',
 '7T MRI',
 'shower',
 '3TMRI',
 'shower',
 'shower',
 'shower',
 'shower end ???',
 'MRI',
 '3T end??',
 'shower end??',
 '3T correct date??',
 '7T',
 'shower end??',
 '7T',
 'MEG']

In [14]:
start_mks = [mk_idx['log'][x] for x in range(0,2,len(mk_idx['log']))]

In [30]:
len(mc_dict['log']['idxs'])

20

In [31]:
len(mk_idx['log'])

40

In [32]:
importlib.reload(act)
act.write_Mtimes_new(clip_dat,mk_idx,os.path.join(out_dir,sub_long),mc_dict)

M
15
z
311
m
14
cal
23
23
log
20
20


,OffDate,OffTime,OnDate,OnTime,marker,Comment
0,08-Feb-16,04:49 PM,08-Feb-16,05:03 PM,z,
1,08-Feb-16,06:33 PM,08-Feb-16,07:01 PM,z,
2,08-Feb-16,07:07 PM,08-Feb-16,07:45 PM,z,
0,08-Feb-16,09:30 PM,09-Feb-16,07:00 AM,cal,Sleep study
3,08-Feb-16,10:05 PM,08-Feb-16,10:43 PM,z,
0,09-Feb-16,12:27 AM,09-Feb-16,12:47 AM,m,
0,09-Feb-16,12:42 AM,09-Feb-16,01:20 AM,log,shower
0,09-Feb-16,12:45 AM,09-Feb-16,01:48 PM,M,
4,09-Feb-16,12:45 AM,09-Feb-16,01:23 AM,z,
5,09-Feb-16,01:45 AM,09-Feb-16,08:33 AM,z,


In [33]:
if 'M' in mk_idx.keys():
    del mk_idx['M']

act.write_dat(clip_dat,mk_idx,os.path.join(out_dir,sub_long))

,DateTime,activity,M,z,m,cal,log
0,08-Feb-16 03:40 PM,81,,0,0,0,0
1,08-Feb-16 03:41 PM,0,,0,0,0,0
2,08-Feb-16 03:42 PM,35,,0,0,0,0
3,08-Feb-16 03:43 PM,170,,0,0,0,0
4,08-Feb-16 03:44 PM,351,,0,0,0,0
5,08-Feb-16 03:45 PM,755,,0,0,0,0
6,08-Feb-16 03:46 PM,222,,0,0,0,0
7,08-Feb-16 03:47 PM,194,,0,0,0,0
8,08-Feb-16 03:48 PM,159,,0,0,0,0
9,08-Feb-16 03:49 PM,0,,0,0,0,0


## Make graphs

In [36]:
coms=[]
idxs=[]
for mm in mc_dict.keys():
    print(mm)
    idxs= idxs +(mc_dict[mm]['idxs'])
    coms= coms + (mc_dict[mm]['comments'])
zipped_pairs = zip(idxs,coms)
gc_coms = [x for _, x in sorted(zipped_pairs)]
gc_idx = np.sort(idxs)

gc=[np.asarray(gc_idx),gc_coms]

cal
log


In [37]:
gc

[array([  350,   542,  1325,  1340,  1905,  2540,  3095,  3230,  3920,
         4280,  4280,  4670,  5300,  5305,  5770,  6860,  6870, 11800,
        16375, 17150, 18380, 18950, 19110, 19990, 20000, 20510, 20600,
        21260, 22700, 23390, 24080, 24440, 24830, 26060, 26060, 27020,
        27850, 40160, 40670, 41655, 41660, 42860, 42860]),
 ['Sleep study',
  'shower',
  '7T MRI',
  '7T',
  'shower',
  'MEG',
  'shower',
  'Sleep study',
  'Infusion',
  'MEG',
  'MEG',
  'Sleep study',
  '7T',
  '7T MRI',
  'shower',
  '3T',
  '3TMRI',
  'shower',
  'shower',
  'shower',
  '7T',
  'shower end ???',
  'MRI',
  '3T end??',
  '3T',
  'Sleep study',
  'shower end??',
  '3T correct date??',
  'MEG',
  'Sleep study',
  'Infusion',
  'MEG',
  'Sleep study',
  '7T',
  '7T',
  '3T',
  'shower end??',
  '3T',
  'Sleep study',
  '7T',
  '7T',
  'MEG',
  'MEG']]

In [ ]:
importlib.reload(act)

act.plot_awd(clip_dat,{'cal':mk_idx['cal'],'log':mk_idx['log']},max_act=500,show=False,comments=gc,fn_pref=os.path.join(out_dir,sub+'_M+logs'),plot_type='single',debug=True)


32
08-Feb-16
0 499
input 0 [-350]
plot this [-350, 499]
input 1 []
[0]
09-Feb-16
500 1939
input 0 [920, -1340, 1460]
plot this [-500, 920, -1340, 1460]
input 1 [-542, 580, -1325, 1495, -1905]
plot this [-542, 580, -1325, 1495, -1905, 1939]
[1 2 3 4]
10-Feb-16
1940 3379
input 0 [-2540, 2660, -3230]
plot this [-2540, 2660, -3230, 3379]
input 1 [1945, -3095, 3147]
plot this [-1940, 1945, -3095, 3147]
[5 6 7]
11-Feb-16
3380 4819
input 0 [3800, -3920, 3960, -4280, 4400, -4670]
plot this [-3380, 3800, -3920, 3960, -4280, 4400, -4670, 4819]
input 1 [-4280, 4400]
plot this [-4280, 4400]
[ 8  9 10 11]
12-Feb-16
4820 6259
input 0 [5240, -5300, 5420]
plot this [-4820, 5240, -5300, 5420]
input 1 [-5305, 5410, -5770, 5835]
plot this [-5305, 5410, -5770, 5835]
[12 13 14]
13-Feb-16
6260 7699
input 0 [-6860, 6950]
plot this [-6860, 6950]
input 1 [-6870, 6965]
plot this [-6870, 6965]
[15 16]
14-Feb-16
7700 9139
input 0 []
input 1 []
[]
15-Feb-16
9140 10579
input 0 []
input 1 []
[]
16-Feb-16
10580 12019

In [ ]:
importlib.reload(act)

act.plot_awd(clip_dat,{'cal':mk_idx['cal'],'log':mk_idx['log']},max_act=250,show=False,comments=gc,fn_pref=os.path.join(out_dir,sub+'_M+logs_zoom'),plot_type='single',debug=True)


In [ ]:
      mm_dt = [ awd_dat['dt_list'][ii] for ii in mk_idx['m'] ]


In [ ]:
mm_dt

In [ ]:
mk_idx['log']

In [ ]:
clip_dat['dt_list'][542]